# CAE-LSTM降阶模型

## 环境安装

本案例要求 **MindSpore >= 2.0.0** 版本以调用如下接口: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*。具体请查看[MindSpore安装](https://www.mindspore.cn/install)。

此外，你需要安装 **MindFlow >=0.1.0** 版本。如果当前环境还没有安装，请按照下列方式选择后端和版本进行安装。

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 概述

为有效降低使用CFD方法的设计成本和周期，近年来降阶模型得到了广泛的关注。对于复杂的可压缩流动，使用本征正交分解(POD)等线性方法进行流场降维，需要大量的模态才能保证流场重建的精度。已有研究表明，采用非线性降维方法能够有效减少所需模态数。卷积自编码器(CAE)是一种由编码器和解码器组成的神经网络，能够实现数据降维和重构，可看作是POD方法的非线性拓展。采用CAE进行流场数据的非线性降维，同时使用长短期记忆神经网络(LSTM)进行流场状态的时间演化。针对非定常可压缩流动，CAE-LSTM降阶模型能够在使用较少自由变量数的前提下获得较高的重构和预测精度。

## 模型架构

CAE-LSTM降阶模型采用CAE网络对流场进行降维，提取流动数据的特征，将其压缩到编码器的隐藏空间中，再用LSTM网络对隐藏空间的自由变量进行系数时间演化，得到流动其他时刻的自由变量，再通过CAE网络的解码器将演化的自由变量进行解码，重建得到相应时刻的流场流动数据。CAE-LSTM流动降阶模型的构造依赖于CAE网络的数据降维和LSTM网络的系数时间演化。与现有的POD/DMD等方法相比，使用CAE网络对流场数据进行非线性降维，同时使用LSTM网络对自由变量进行无方程演化，可以在保证流场降阶模型具备一定精度的情况下，得到更高的压缩比，提高流场预测的效率。

+ 输入：输入一段时间的流场。
+ 压缩：通过CAE的编码器对流场进行降维，提取高维时空流动特征。
+ 演化：通过LSTM学习低维空间流场时空特征的演变，预测下一时刻。
+ 重建：通过CAE的解码器将预测的流场低维特征恢复到高维空间。
+ 输出：输出对下一时刻瞬态流场的预测结果。

训练时，首先进行CAE网络的训练，训练完成之后使用CAE的编码器得到流场的低维特征，将此低维特征作为LSTM网络的数据集，进行LSTM网络的训练。

![CAE-LSTM.png](./images/cae_lstm_CN.png)

## 训练环境

导入训练所需函数库，其中`src`文件夹包括数据集处理函数、网络模型和训练loss可视化函数。

训练可选择不同的算例：`sod`，`shu_osher`，`riemann`，`kh`和`cylinder`，其中`sod`和`shu_osher`为一维算例，`riemann`，`kh`和`cylinder`为二维算例。在`parser.add_argument`的`case`选择中修改算例名称即可运行相应的算例。如若使用命令行调用网络训练，也可在`--case`后填写算例名称运行相应算例。默认选择`sod`算例。

训练默认采用Mindspore框架的静态图模式(GRAPH)，在GPU(默认)或Ascend进行训练(单卡)。

In [1]:
import os
import time
import argparse

import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config
from src import create_cae_dataset, CaeNet1D, CaeNet2D, plot_train_loss

In [2]:
np.random.seed(0)
set_seed(0)

In [3]:
parser = argparse.ArgumentParser(description='CaeNet')
parser.add_argument("--case", type=str, default="sod", choices=["sod", "shu_osher", "riemann", "kh"],
                    help="Which case to run, support 'sod', 'shu_osher', 'riemann', 'kh'")
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(case=args.case,
                    mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id,
                    config_file_path=args.config_file_path)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## CAE网络训练参数设置

根据所选取的算例，从config.yaml文件里导入相应的数据集、CAE模型和优化器的参数配置。对于二维圆柱绕流算例，在config.yaml文件中需注明算例的具体雷诺数，以读取相应雷诺数下的数据进行训练。

In [4]:
config = load_yaml_config(args.config_file_path)
if args.case == 'sod' or args.case == 'shu_osher':
    data_params = config["1D_cae_data"]
    model_params = config["1D_cae_model"]
    optimizer_params = config["1D_cae_optimizer"]
else:
    data_params = config["2D_cae_data"]
    model_params = config["2D_cae_model"]
    optimizer_params = config["2D_cae_optimizer"]

训练过程loss文件保存路径默认为optimizer_params["summary_dir"]，权重参数保存在ckpt文件夹中。

In [5]:
summary_dir = optimizer_params["summary_dir"]
if not os.path.exists(summary_dir):
    os.mkdir(summary_dir)
ckpt_dir = os.path.join(summary_dir, 'ckpt')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## 构建CAE网络

CAE网络由多层卷积和极大池化构成编码器，由多层卷积和上采样构成解码器。使用MSELoss损失函数和Adam优化器。

In [6]:
if args.case == 'sod' or args.case == 'shu_osher':
    cae = CaeNet1D(model_params["data_dimension"], model_params["conv_kernel_size"],
                   model_params["maxpool_kernel_size"], model_params["maxpool_stride"],
                   model_params["encoder_channels"], model_params["decoder_channels"])
else:
    cae = CaeNet2D(model_params["data_dimension"], model_params["conv_kernel_size"],
                   model_params["maxpool_kernel_size"], model_params["maxpool_stride"],
                   model_params["encoder_channels"], model_params["decoder_channels"],
                   model_params["channels_dense"])

loss_fn = nn.MSELoss()
cae_opt = nn.Adam(cae.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])

## CAE网络数据集

数据集下载地址：[data_driven/cae-lstm/dataset](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/cae-lstm)

导入数据集之后进行数据下沉设置。

In [7]:
cae_dataset, _ = create_cae_dataset(data_params["data_path"], data_params["batch_size"], data_params["multiple"])

sink_process = data_sink(train_step, cae_dataset, sink_size=1)
train_data_size = cae_dataset.get_dataset_size()

## CAE网络模型训练

搭建forward_fn和train_step，开始CAE网络的训练，并将训练loss可视化。

In [8]:
def forward_fn(data, label):
    logits = cae(data)
    loss = loss_fn(logits, label)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, cae_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    loss = ops.depend(loss, cae_opt(grads))
    return loss

print(f"====================Start CaeNet train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    cae.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(cae, f"{ckpt_dir}/cae_{epoch}.ckpt")
print(f"=====================End CaeNet train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "cae")

pid:23104
====================Start CaeNet train=======================
epoch: 1 train loss: 0.00859989 epoch time: 3.23s
epoch: 2 train loss: 0.00563688 epoch time: 0.52s
epoch: 3 train loss: 0.00485115 epoch time: 0.53s
epoch: 4 train loss: 0.00341164 epoch time: 0.62s
epoch: 5 train loss: 0.00332990 epoch time: 0.57s
...
epoch: 4396 train loss: 3.69731242e-06 epoch time: 0.51s
epoch: 4397 train loss: 2.65247831e-06 epoch time: 0.55s
epoch: 4398 train loss: 1.14417275e-06 epoch time: 0.54s
epoch: 4399 train loss: 4.97764995e-06 epoch time:0.52s
epoch: 4400 train loss: 2.48092419e-06 epoch time: 0.55s
====================End CaeNet train=======================


## CAE流场重建结果

在训练完CAE网络后，可运行`cae_eval.py`查看CAE的训练结果，以判断是否继续进行LSTM网络的训练。

下图分别为五个算例的真实流场，CAE流场重建结果以及它们之间的误差曲线。对于前四个算例，前两个流场结果展现了流场中不同位置的密度随时间的变化情况，第三个误差曲线展现了CAE重建流场和真实流场label的平均相对误差随时间的变化情况。对于圆柱绕流算例，以雷诺数Re = 300状态为例进行训练（在`config.yaml`里将数据集选为`cylinder_re300.npy`），预测推理时使用其他雷诺数（以雷诺数Re = 200状态为例，在`config.yaml`里将数据集选为`cylinder_re200.npy`）下的流场，前两个流场结果展现了流场中不同位置的流向速度随时间的变化情况，第三个误差曲线展现了CAE重建流场和真实流场label的绝对误差随时间的变化情况。误差满足流场重建精度需求。

Sod激波管：

<figure class="harf">
    <img src="./images/sod_cae_reconstruction.gif" title="sod_cae_reconstruction" width="600"/>
    <img src="./images/sod_cae_error.png" title="sod_cae_error" width="300"/>
</center>

Shu_Osher问题：

<figure class="harf">
    <img src="./images/shu_osher_cae_reconstruction.gif" title="shu_osher_cae_reconstruction" width="600"/>
    <img src="./images/shu_osher_cae_error.png" title="shu_osher_cae_error" width="300"/>
</center>

黎曼问题：

<figure class="harf">
    <img src="./images/riemann_cae_reconstruction.gif" title="riemann_cae_reconstruction" width="600"/>
    <img src="./images/riemann_cae_error.png" title="riemann_cae_error" width="300"/>
</center>

开尔文亥姆霍兹不稳定性问题：

<figure class="harf">
    <img src="./images/kh_cae_reconstruction.gif" title="kh_cae_reconstruction" width="600"/>
    <img src="./images/kh_cae_error.png" title="kh_cae_error" width="300"/>
</center>

圆柱绕流（Re = 200）：

<figure class="harf">
    <img src="./images/cylinder_cae_reconstruction.gif" title="cylinder_cae_reconstruction" width="600"/>
    <img src="./images/cylinder_cae_error.png" title="cylinder_cae_error" width="300"/>
</center>

## LSTM网络框架及训练设置

LSTM网络框架搭建、训练环境等相关处理与CAE网络类似。

首先导入训练所需函数库，然后导入相应算例的LSTM网络数据集设置参数、LSTM模型和优化器参数设置。默认训练loss保存路径为optimizer_params["summary_dir"]，权重参数保存在ckpt文件夹下。网络由多层LSTM层和一层全连接层组成，使用MSELoss损失函数和Adam优化器。

In [9]:
import os
import time
import argparse

import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config
from src import create_lstm_dataset, Lstm, plot_train_loss
from cae_eval import cae_eval

np.random.seed(0)
set_seed(0)

parser = argparse.ArgumentParser(description='Lstm')
parser.add_argument("--case", type=str, default="sod", choices=["sod", "shu_osher", "riemann", "kh"],
                    help="Which case to run, support 'sod', 'shu_osher', 'riemann', 'kh'")
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(case=args.case,
                    mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id,
                    config_file_path=args.config_file_path)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

# prepare params
config = load_yaml_config(args.config_file_path)
if args.case == 'sod' or args.case == 'shu_osher':
    data_params = config["1D_lstm_data"]
    model_params = config["1D_lstm_model"]
    optimizer_params = config["1D_lstm_optimizer"]
else:
    data_params = config["1D_lstm_data"]
    model_params = config["1D_lstm_model"]
    optimizer_params = config["1D_lstm_optimizer"]

# prepare summary file
summary_dir = optimizer_params["summary_dir"]
ckpt_dir = os.path.join(summary_dir, 'ckpt')

# prepare model
lstm = Lstm(model_params["latent_size"], model_params["hidden_size"], model_params["num_layers"])
loss_fn = nn.MSELoss()
lstm_opt = nn.Adam(lstm.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])

## LSTM网络数据集加载与处理

LSTM网络数据集由CAE的编码器得到，创建数据集之后进行数据下沉设置。

In [10]:
# prepare dataset
latent_true = cae_eval(args.config_file_path, args.case)
lstm_dataset, _ = create_lstm_dataset(latent_true, data_params["batch_size"], data_params["time_size"],
                                      data_params["latent_size"], data_params["time_window"],
                                      data_params["gaussian_filter_sigma"])

# data sink
sink_process = data_sink(train_step, lstm_dataset, sink_size=1)
train_data_size = lstm_dataset.get_dataset_size()

## LSTM网络模型训练

搭建forward_fn和train_step，开始LSTM网络的训练，并将训练loss可视化。

In [11]:
# Define forward function
def forward_fn(data, label):
    logits = lstm(data)
    loss = loss_fn(logits, label)
    return loss

# Get gradient function
grad_fn = ops.value_and_grad(forward_fn, None, lstm_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    loss = ops.depend(loss, lstm_opt(grads))
    return loss

print(f"====================Start Lstm train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    lstm.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(lstm, f"{ckpt_dir}/lstm_{epoch}.ckpt")
print(f"=====================End Lstm train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "lstm")

pid:22152
====================Start Lstm train=======================
epoch: 1 train loss: 0.4425844 epoch time: 3.75s
epoch: 2 train loss: 0.23611887 epoch time: 0.75s
epoch: 3 train loss: 0.65945524 epoch time: 0.76s
epoch: 4 train loss: 0.77271056 epoch time: 0.80s
epoch: 5 train loss: 0.3535387 epoch time: 0.81s
...
epoch: 4396 train loss: 9.665465e-05 epoch time: 0.76s
epoch: 4397 train loss: 5.5045904e-05 epoch time: 0.77s
epoch: 4398 train loss: 0.00013155791 epoch time: 0.80s
epoch: 4399 train loss: 0.0769522e-05 epoch time: 0.80s
epoch: 4400 train loss: 0.0010389996 epoch time: 0.80s
====================End Lstm train=======================


## 预测流场结果可视化

运行`cae_lstm_eval.py`可查看CAE-LSTM降阶模型的预测结果。

下图分别为五个不同算例的真实流场，CAE-LSTM网络的预测结果和相对应的平均误差。其中圆柱绕流算例的训练使用雷诺数Re = 300的数据集，推理预测使用雷诺数Re = 200的数据集。整个预测时间误差满足流场预测精度需求。

Sod激波管：

<figure class="harf">
    <img src="./images/sod_cae_lstm_predict.gif" title="sod_cae_lstm_predict" width="600"/>
    <img src="./images/sod_cae_lstm_error.png" title="sod_cae_lstm_error" width="300"/>
</center>

Shu_osher问题：

<figure class="harf">
    <img src="./images/shu_osher_cae_lstm_predict.gif" title="shu_osher_cae_lstm_predict" width="600"/>
    <img src="./images/shu_osher_cae_lstm_error.png" title="shu_osher_cae_lstm_error" width="300"/>
</center>

黎曼问题：

<figure class="harf">
    <img src="./images/riemann_cae_lstm_predict.gif" title="riemann_cae_lstm_predict" width="600"/>
    <img src="./images/riemann_cae_lstm_error.png" title="riemann_cae_lstm_error" width="300"/>
</center>

开尔文亥姆霍兹不稳定性问题：

<figure class="harf">
    <img src="./images/kh_cae_lstm_predict.gif" title="kh_cae_lstm_predict" width="600"/>
    <img src="./images/kh_cae_lstm_error.png" title="kh_cae_lstm_error" width="300"/>
</center>

圆柱绕流（Re = 200）：

<figure class="harf">
    <img src="./images/cylinder_cae_lstm_predict.gif" title="cylinder_cae_lstm_predict" width="600"/>
    <img src="./images/cylinder_cae_lstm_error.png" title="cylinder_cae_lstm_error" width="300"/>
</center>